In [62]:
import pandas as pd
import numpy as np

In [5]:
spam_data = pd.read_csv("./SMS_Spam_Data.csv", encoding='latin-1')

In [6]:
spam_data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
spam_data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [9]:
from string import punctuation
punctuations = punctuation

from nltk.corpus import stopwords
stop = stopwords.words("english")

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def _clean(text):
    
    text = text.lower()
    
    text = "".join(t for t in text if t not in punctuations)
    
    words = text.split()
    
    words = [w for w in words if w not in stop]
    
    words = [lemmatizer.lemmatize(w, "v") for w in words]
    words = [lemmatizer.lemmatize(w, "n") for w in words]

    
    return " ".join(words)

_clean("I will be playing cricket today!!!")

'play cricket today'

In [10]:
spam_data["cleaned_text"] = spam_data.text.apply(_clean)

In [11]:
spam_data

,label,text,cleaned_text
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,2nd time try 2 contact u u ã¥â£750 pound prize...
5568,ham,Will Ã_ b going to esplanade fr home?,ã b go esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood soany suggestion
5570,ham,The guy did some bitching but I acted like i'd...,guy bitch act like id interest buy something e...


# Feature Enginerring

In [17]:
## Adding Meta Features ea. word count, char count

spam_data["word_count"] = spam_data.text.apply(lambda x: len(x.split()))
spam_data["word_count_cleaned"] = spam_data.cleaned_text.apply(lambda x: len(x.split()))

spam_data["char_count"] = spam_data.text.apply(lambda x: len(x))
spam_data["char_count_without_space"] = spam_data.text.apply(lambda x: len(x.replace(' ','')))

spam_data["number_of_digit"] = spam_data.cleaned_text.apply(lambda x: sum([1 if  w.isdigit() else 0 for w in x.split()]))

In [18]:
spam_data

,label,text,cleaned_text,word_count,word_count_cleaned,char_count,char_count_without_space,number_of_digit
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,111,92,0
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,6,6,29,24,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,155,128,3
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,49,39,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though,13,8,61,49,0
...,...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,2nd time try 2 contact u u ã¥â£750 pound prize...,30,20,163,134,3
5568,ham,Will Ã_ b going to esplanade fr home?,ã b go esplanade fr home,8,6,38,31,0
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood soany suggestion,10,4,57,48,0
5570,ham,The guy did some bitching but I acted like i'd...,guy bitch act like id interest buy something e...,26,14,125,100,0


In [27]:
pos_dic = {"noun": ["NNP", "NN", "NNS", "NNPS"], "verb":["VBZ", "VB","VBD","VBN"]}
import nltk
def pos_check (txt, family):
    tags = nltk.pos_tag(nltk.word_tokenize(txt))
    count =0
    for tag in tags:
        tag = tag[1]
        if tag in pos_dic[family]:
            count += 1
    return count

pos_check("They are playing in the ground","noun")

1

In [28]:
spam_data["noun_count"] = spam_data.text.apply(lambda x:pos_check(x, "noun"))
spam_data["verb_count"] = spam_data.text.apply(lambda x:pos_check(x, "verb"))

In [29]:
spam_data

,label,text,cleaned_text,word_count,word_count_cleaned,char_count,char_count_without_space,number_of_digit,noun_count,verb_count
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,111,92,0,10,1
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,6,6,29,24,0,4,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,155,128,3,13,3
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,49,39,0,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though,13,8,61,49,0,1,4
...,...,...,...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,2nd time try 2 contact u u ã¥â£750 pound prize...,30,20,163,134,3,9,5
5568,ham,Will Ã_ b going to esplanade fr home?,ã b go esplanade fr home,8,6,38,31,0,3,2
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood soany suggestion,10,4,57,48,0,4,1
5570,ham,The guy did some bitching but I acted like i'd...,guy bitch act like id interest buy something e...,26,14,125,100,0,5,4


In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cvc = CountVectorizer()
cvc.fit(spam_data.cleaned_text.values)
word_count = cvc.transform(spam_data.cleaned_text)

In [31]:
word_count

<5572x8206 sparse matrix of type '<class 'numpy.int64'>'
	with 46827 stored elements in Compressed Sparse Row format>

In [40]:
tf_idf_score = TfidfVectorizer(max_features=300)
tf_idf_score.fit(spam_data.cleaned_text.values)
tf_idf_matrix = tf_idf_score.transform(spam_data.cleaned_text.values)

In [41]:
tf_idf = dict(zip(tf_idf_score.get_feature_names(), tf_idf_score.idf_))
tf_idf =  pd.DataFrame().from_dict(tf_idf, orient="index")
tf_idf.columns = ["tf_idf"]

In [42]:
tf_idf

,tf_idf
100,5.936809
1000,5.754488
150,5.888019
150ppm,6.070341
16,5.693863
...,...
yes,5.159781
yet,5.713666
yo,6.099328
youre,5.655397


In [43]:
from scipy.sparse import hstack, csr_matrix

meta_features = ["word_count", "word_count_cleaned", "char_count","char_count_without_space","number_of_digit","noun_count","verb_count"]

feature_set_1 = spam_data[meta_features]

train = hstack([tf_idf_matrix, csr_matrix(feature_set_1)], "csr")

In [82]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target = le.fit_transform(spam_data["label"])


In [83]:
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split

token = text.Tokenizer()
token.fit_on_texts(spam_data['text'])
word_index = token.word_index

train_X, valid_X, train_y, valid_y = train_test_split(spam_data["text"], target)

train_X = sequence.pad_sequences(token.texts_to_sequences(train_X) , maxlen=70)
valid_X =sequence.pad_sequences(token.texts_to_sequences(valid_X), maxlen=70)

embedding_matrix = np.zeros((len(word_index)+1, 300))
for word, index in word_index.items():
    embedding_vector = glove.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [84]:
def train_model(classifier, feature_vector_train, label, feature_vector_vald, valid_y):
    classifier.fit(feature_vector_train, label)
    predictions = classifier.predict(feature_vector_vald)
    predictions = predictions.argmax(axis = -1)
    return accuracy_score(predictions, valid_y)

In [58]:
import pickle

with open('../../Word_Embeddings_in_Action_-_GloVe/Word Embeddings in Action - GloVe/glove.6B.300d.pkl', 'rb') as fp:
    # load glove from disk 
    glove = pickle.load(fp)

In [87]:
from keras import layers, models, optimizers
from sklearn.metrics import accuracy_score

def create_cnn():
    
    input_layer = layers.Input((70, ))
    
    embedding_layer = layers.Embedding(len(word_index)+1, 300, weights = [embedding_matrix], trainable = False)(input_layer)
    
    conv_layer = layers.Convolution1D(100, 3, activation ='relu')(embedding_layer)
    
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    output_layer = layers.Dense(50, activation= "relu")(pooling_layer)
    
    output_layer = layers.Dropout(0.25)(output_layer)
    
    output_layer = layers.Dense(1, activation= "sigmoid")(output_layer)
    
    model = models.Model(inputs = input_layer, outputs = output_layer)
    
    model.compile(optimizer=optimizers.Adam(), loss= "binary_crossentropy" )
    
    return model
    

In [88]:
classifier = create_cnn()
train_model(classifier, train_X,train_y,valid_X, valid_y)

Epoch 1/1
4179/4179 [==============================] - 8s 2ms/step - loss: 0.2151


0.8693467336683417